In [1]:
import sys

sys.path.append('../../src')
import pandas
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve

In [2]:
# ======================================================================================================================
# Initialize Case Study
# ======================================================================================================================
_time_intervals = 10  # Number of time intervals in a planning horizon    (L_chi)
_commodities = 2  # Number of commodities                             (rho)
_exec_scenarios = 4  # Number of execution scenarios                     (chi)

M = 1e6  # Big M
availability_factor = pandas.DataFrame(data={'com1_pur': [1, 0.5, 0, 1], 'com2_pur': [1, 0.5, 1, 1]})
# Define temporal scales
scales = TemporalScale(discretization_list=[_exec_scenarios, _time_intervals])

In [3]:
# ======================================================================================================================
# Declare resources/commodities
# ======================================================================================================================

com1_pur = Resource(name='com1_pur', cons_max=200, block={'imp': 1, 'urg': 1}, price=50,
                    label='Commodity 1 consumed from outside the system', varying=[VaryingResource.DETERMINISTIC_AVAILABILITY])

com1 = Resource(name='com1', block={'imp': 1, 'urg': 1}, label='Commodity 1')

com1_sold = Resource(name='com1_sold', revenue=100.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')

com2_pur = Resource(name='com2_pur', cons_max=200, block={'imp': 1, 'urg': 1}, price=50,
                    label='Commodity 2 consumed from outside the system', varying=[VaryingResource.DETERMINISTIC_AVAILABILITY])

com2 = Resource(name='com2', block={'imp': 1, 'urg': 1}, label='Commodity 2')

com2_sold = Resource(name='com2_sold', revenue=100.00, demand=True, sell=True, label='Commodity 2 sold to outside the system')

In [4]:
# ======================================================================================================================
# Declare processes/storage capacities
# ======================================================================================================================
com1_procure = Process(name='procure com1', prod_max=M, conversion={com1_pur: -1, com1: 1}, capex=0, vopex=0, fopex=0,
                  label='Procure com1')
com1_sell = Process(name='sell com1', prod_max=M, conversion={com1: -1, com1_sold: 1}, capex=0, vopex=0, fopex=0,
                  label='Sell com1')

com1_store10 = Process(name='store10', storage=com1, store_max=10, prod_max=M, capex=100, vopex=1,
                  label="Storage capacity of 10 units")
com1_store20 = Process(name='store20', storage=com1, store_max=20, prod_max=M, capex=200, vopex=2,
                  label="Storage capacity of 20 units")
com1_store50 = Process(name='store50', storage=com1, store_max=50, prod_max=M, capex=500, vopex=5,
                  label="Storage capacity of 50 units")

com2_procure = Process(name='procure com2', prod_max=M, conversion={com2_pur: -1, com2: 1}, capex=0, vopex=0, fopex=0,
                  label='Procure com1')
com2_sell = Process(name='sell com2', prod_max=M, conversion={com2: -1, com2_sold: 1}, capex=0, vopex=0, fopex=0,
                  label='Sell com1')

com2_store10 = Process(name='store101', storage=com2, store_max=10, prod_max=M, capex=100, vopex=1,
                  label="Storage capacity of 10 units")
com2_store20 = Process(name='store201', storage=com2, store_max=20, prod_max=M, capex=200, vopex=2,
                  label="Storage capacity of 20 units")
com2_store50 = Process(name='store501', storage=com2, store_max=50, prod_max=M, capex=500, vopex=5,
                  label="Storage capacity of 50 units")

In [5]:
# ======================================================================================================================
# Declare locations/warehouses
# ======================================================================================================================
loc1 = Location(name='loc1', processes={com1_procure, com1_store20, com2_procure, com2_store20}, label="Location 1", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0, availability_factor={com1_pur: availability_factor['com1_pur'], com2_pur: availability_factor['com2_pur']})
loc2 = Location(name='loc2', processes={com1_store20, com2_store20}, label="Location 2", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0)
loc3 = Location(name='loc3', processes={com1_store20, com2_store20}, label="Location 3", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0)
loc4 = Location(name='loc4', processes={com1_store50, com2_store50}, label="Location 4", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0)
loc5 = Location(name='loc5', processes={com1_store50, com1_sell, com2_store50, com2_sell}, label="Location 5", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0)
loc6 = Location(name='loc6', processes={com1_procure, com1_store10, com2_procure, com2_store10}, label="Location 6", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0, availability_factor={com1_pur: availability_factor['com1_pur'], com2_pur: availability_factor['com2_pur']})
loc7 = Location(name='loc7', processes={com1_store10, com2_store10}, label="Location 7", scales=scales, demand_scale_level=1,
                capacity_scale_level=0, availability_scale_level=0)

C:\Users\shivam.vedant\PycharmProjects\energiapy\src\energiapy\components\location.py:149: UserWarning: Input should be a dict of a DataFrame, Dict[Resource, float]
  warn(


In [6]:
# ======================================================================================================================
# Declare transport/trucks
# ======================================================================================================================
truck20 = Transport(name='truck20', resources=[com1, com2], trans_max=20, label='Truck with maximum capacity of 20 units',
                    trans_cost=0.02)
truck30 = Transport(name='truck30', resources=[com1, com2], trans_max=30, label='Truck with maximum capacity of 30 units',
                    trans_cost=0.03)
truck40 = Transport(name='truck40', resources=[com1, com2], trans_max=40, label='Truck with maximum capacity of 40 units',
                    trans_cost=0.04)
truck50 = Transport(name='truck50', resources=[com1, com2], trans_max=50, label='Truck with maximum capacity of 50 units',
                    trans_cost=0.05)
truck100 = Transport(name='truck100', resources=[com1, com2], trans_max=100, label='Truck with maximum capacity of 100 units',
                     trans_cost=0.1)

In [7]:
transport_matrix = [
    [[], [truck50], [truck30], [], [], [], []],  # sink: location 1
    [[truck50], [], [], [truck50], [truck30], [], []],  # sink: location 2
    [[truck30], [], [], [truck20], [], [], []],  # sink: location 3
    [[], [truck50], [truck20], [], [truck100], [truck50], [truck40]],  # sink: location 4
    [[], [truck30], [], [truck100], [], [], [truck40]],  # sink: location 5
    [[], [], [], [truck50], [], [], []],  # sink: location 6
    [[], [], [], [truck40], [truck40], [], []]  # sink: location 7
]

distance_matrix = [
    [0, 55, 196, M, M, M, M],
    [55, 0, M, 163, 112, M, 134],
    [196, M, 0, 63, M, M, M],
    [M, 163, 63, 0, 95, 117, 88],
    [M, 112, M, 95, 0, M, 134],
    [M, M, M, 117, M, 0, M],
    [M, 134, M, 88, 134, M, 0]
]

# ======================================================================================================================
# Declare network
# ======================================================================================================================
locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7]

sources = list(locset)
sinks = list(locset)

network = Network(name='Network', source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix,
                  distance_matrix=distance_matrix)

In [8]:
demand_dict = {i: {com1_sold: 100, com2_sold: 100} if i == loc5 else {com1_sold: 0, com2_sold: 0} for i in locset}
demand_penalty_dict = {i: {com1_sold: 125, com2_sold: 125} if i == loc5 else {com1_sold: 0, com2_sold: 0} for i in locset}

scenario = Scenario(name='scenario', scales=scales, scheduling_scale_level=1, network_scale_level=0,
                    purchase_scale_level=0, availability_scale_level=0, demand_scale_level=1,
                    network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='scenario')

In [9]:
scenario.availability_factor.keys()

dict_keys(['loc7', 'loc6', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5'])

In [10]:
 problem = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.TRANSPORT,
                                                    Constraints.RESOURCE_BALANCE, Constraints.PRODUCTION,
                                                    Constraints.INVENTORY, Constraints.DEMAND}, demand_sign='eq',
                    objective=Objective.PROFIT_W_DEMAND_PENALTY)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint production mode
constraint inventory balance
ERROR: Rule failed when generating expression for Constraint
constraint_resource_consumption_varying with index ('loc1', 'com1_pur', 0, 0):
AttributeError: 'str' object has no attribute 'name'
ERROR: Constructing component 'constraint_resource_consumption_varying' from
data=None failed: AttributeError: 'str' object has no attribute 'name'


AttributeError: 'str' object has no attribute 'name'

In [14]:
scenario.capacity_factor

{'loc7': None,
 'loc6': None,
 'loc1': None,
 'loc2': None,
 'loc3': None,
 'loc4': None,
 'loc5': None}

In [ ]:
results = solve(scenario=scenario, instance=problem, solver='gurobi', name='MILP')

In [ ]:
print([(key, value) for key, value in results.output['S'].items() if value != 0])

In [ ]:
print([(key, value) for key, value in results.output['Inv'].items() if value != 0])

In [ ]:
print([(key, value) for key, value in results.output['Demand_penalty'].items() if value != 0])

In [ ]:
plot_location.availability_factor(resource=com1_pur,location=loc6,color='green')

In [ ]:
plot_results.schedule(results=results, y_axis='Inv', component='store20_com1_stored', location='loc3', color='green')

In [ ]:
plot_results.schedule(results=results, y_axis='Inv', component='store10_com1_stored', location='loc7', color='green')

In [ ]:
plot_results.schedule(results=results, y_axis='Inv', component='store50_com1_stored', location='loc4', color='green')

In [ ]:
plot_results.schedule(results=results, y_axis='S', component='com1_sold', location='loc5', color='green')

In [ ]:
plot_results.schedule(results=results, y_axis='S', component='com2_sold', location='loc5', color='green')

In [ ]:
availability_factor

In [ ]:
total_availability_factor = availability_factor['com2_pur']
total_availability_factor